In [18]:
import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv('DuvalSFR.csv')

In [20]:
print(df.shape)
df.head()

(252463, 320)


,RE,owner_1,owner_2,owner_3,mailing_address,mailing_city,mailing_state,property_use,subdivision,neighborhood,...,sub19_effecarea,sub19_heatedarea,sub20,sub20_actualarea,sub20_effecarea,sub20_heatedarea,sub21,sub21_actualarea,sub21_effecarea,sub21heated_area
0,000006 0030,NADELEN ROBERT CARL JR,NADELEN ELIZABETH ANN,NaN,2503 SUMMERFIELD LN,JACKSONVILLE,FL,100,SECTION LAND,417300.2,...,0,0,NaN,0,0,0,NaN,0,0,0
1,000007 0010,BRYCEVILLE LAND LLC,NaN,NaN,PO BOX 23637,JACKSONVILLE,FL,100,SECTION LAND,417300.2,...,0,0,NaN,0,0,0,NaN,0,0,0
2,000009 0100,ENGEL ROBERT W,ENGEL GEORGEANN A,NaN,2495 U S 301 HWY N,BALDWIN,FL,100,SECTION LAND,417300.2,...,0,0,NaN,0,0,0,NaN,0,0,0
3,000011 0000,BYRD FAMILY TRUST,BYRD JOEY B,BYRD LISA R,2425 US HWY 301 N,BALDWIN,FL,100,SECTION LAND,417300.2,...,0,0,NaN,0,0,0,NaN,0,0,0
4,000012 0010,WHITTY MICHAEL S,WHITTY DEBORAH B,NaN,2204 US HIGHWAY 301 N,JACKSONVILLE,FL,100,SECTION LAND,417300.2,...,0,0,NaN,0,0,0,NaN,0,0,0


In [21]:
for column in df.columns:
    print(column, '|', df[column].dtype, '|', df[column].count())

RE | object | 252463
owner_1 | object | 252463
owner_2 | object | 126671
owner_3 | object | 10736
mailing_address | object | 252463
mailing_city | object | 252463
mailing_state | object | 252463
property_use | int64 | 252463
subdivision | object | 252463
neighborhood | float64 | 252463
val_method | int64 | 252463
cap_base_yr | int64 | 252463
market_val | int64 | 252463
assessed_val | int64 | 252463
building_val | int64 | 252463
just_val | int64 | 252463
school_taxable | int64 | 252463
county_taxable | int64 | 252463
sjrwnd_taxable | int64 | 252463
taxing_district | object | 252463
square_feet | int64 | 252463
char_1 | object | 20616
char_2 | object | 734
char_3 | object | 3
building | int64 | 252463
type_descr | object | 252463
style | int64 | 252463
class | int64 | 252463
quality | float64 | 252463
actual_yr_built | int64 | 252463
effec_yr_built | int64 | 252463
perc_complete | float64 | 252463
value | int64 | 252463
heated_sf | int64 | 252463
stories | float64 | 252463
bedrooms | flo

In [44]:
# df = df.fillna(value={'owner_2':'NONE','owner_3':'NONE'})
df.neighborhood = df.neighborhood.astype('object')
df.drop(columns=['property_use','perc_complete','restrooms',
                'avg_story_height','reason','improved',
                'landuse_6','landuse_7','landuse_8','landuse_9','landuse_10',
                'zoning_6','zoning_7','zoning_8','zoning_9','zoning_10',
                'unit_type_6','unit_type_7','unit_type_8','unit_type_9','unit_type_10',
                'units_6','units_7','units_8','units_9','units_10',
                'land_val_7','land_val_8','land_val_9','land_val_10',
                'seller_2',
                'feat18','feat19','feat20','feat21','feat22','feat23',
                'feat18_grade','feat19_grade','feat20_grade','feat21_grade','feat22_grade','feat23_grade',
                'feat18_units','feat19_units','feat20_units','feat21_units','feat22_units','feat23_units',
                'feat18_ppu','feat19_ppu','feat20_ppu','feat21_ppu','feat22_ppu','feat23_ppu',
                'feat18_yr_built','feat19_yr_built','feat20_yr_built','feat21_yr_built','feat22_yr_built','feat23_yr_built',
                'feat18_deprec_val','feat19_deprec_val','feat20_deprec_val','feat21_deprec_val','feat22_deprec_val','feat23_deprec_val'
                ],
        inplace=True)
df.shape

(252463, 253)

In [49]:
print(df.landuse_1.unique())

['AGR IV - 1 UNIT PER 2.5 ACRES' 'RES LD 3-7 UNITS PER AC' 'COMMERCIAL'
 'RES RURAL 2 OR LESS UNITS PER AC' 'MIXED-USE PREDOM RES'
 'RES MD 8-19 UNITS PER AC' 'MIXED-USE PREDOM COMMERCIAL'
 'AGR II - 1 UNIT PER 40 ACRES' 'RES NATURAL LD 3-7 UNITS PER AC'
 'RES POND LD 3-7 UNITS PER AC' 'AGR III - 1 UNIT PER 10 ACRES'
 'AGR I - 1 UNIT PER 100 ACRES' 'OFFICE' 'LIGHT INDUSTRIAL'
 'CONSERVATION EASEMENT    ' 'BUSINESS PARK 1'
 'RES NATURAL RURAL 2 OR LESS UTS/AC'
 'RES POND RURAL 2 OR LESS UNITS PER AC'
 'RES RIVER RURAL 2 OR LESS UNITS PER AC' 'RES MARSH LD 3-7 UNITS PER AC'
 'RES RIVER LD 3-7 UNITS PER AC' 'COMM/RES/OFF' 'INSTITUTIONAL'
 'RES CANAL RURAL 2 OR LESS UNITS PER AC' 'RES CANAL LD 3-7 UNITS PER AC'
 'RES MARSH RURAL 2 OR LESS UNITS PER AC' 'HEAVY INDUSTRIAL' 'POND / LAKE'
 'RES/OFFICE' 'RES POND MD 8-19 UNITS PER AC'
 'RES NATURAL MD 8-19 UNITS PER AC' 'SWAMP                    '
 'RES GOLF LD 3-7 UNITS PER AC' 'EXCESS LAND              '
 'RIGHT-OF-WAY & STS       ' 'RES HD 2

In [50]:
l_uses = ['landuse_1', 'landuse_2', 'landuse_3', 'landuse_4', 'landuse_5']
for use in l_uses:
    print(use)
    df = df.fillna(value={use:'NA'})
    for i in range(len(df[use])):
        if df[use][i] != 'NA':
            lst = df[use][i].split()
            w = str()
            for word in lst: 
                w = w + ' ' + word.strip()
            df[use][i] = w.strip()
            if i % 10000 == 0:
                print(i)
sorted(df.landuse_1.unique())
    

landuse_1


<ipython-input-50-2efad9dafb7a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[use][i] = w.strip()


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
landuse_2
240000
landuse_3
240000
landuse_4
landuse_5


['AGR I - 1 UNIT PER 100 ACRES',
 'AGR II - 1 UNIT PER 40 ACRES',
 'AGR III - 1 UNIT PER 10 ACRES',
 'AGR III MARSH - 1 UNIT PER 10 ACRES',
 'AGR IV - 1 UNIT PER 2.5 ACRES',
 'AGR IV CANAL - 1 UNIT PER 2.5 ACRES',
 'AGR IV MARSH - 1 UNIT PER 2.5 ACRES',
 'AGR IV RIVER - 1 UNIT PER 2.5 ACRES',
 'BORROW PIT & DRAIN RESERV',
 'BUSINESS PARK 1',
 'COMM/RES/OFF',
 'COMMERCIAL',
 'COMMERCIAL WATERFRONT',
 'CONSERVATION EASEMENT',
 'EXCESS LAND',
 'HEAVY INDUSTRIAL',
 'INSTITUTIONAL',
 'LIGHT INDUSTRIAL',
 'MARSH',
 'MH MD 8-19 UNITS PER AC',
 'MIXED-USE PREDOM COMMERCIAL',
 'MIXED-USE PREDOM RES',
 'NA',
 'OFFICE',
 'POND / LAKE',
 'RES CANAL LD 3-7 UNITS PER AC',
 'RES CANAL MD 8-19 UNITS PER AC',
 'RES CANAL RURAL 2 OR LESS UNITS PER AC',
 'RES GOLF LD 3-7 UNITS PER AC',
 'RES GOLF MD 8-19 UNITS PER AC',
 'RES HD 20-60 UNITS PER AC',
 'RES LD 3-7 UNITS PER AC',
 'RES MARSH LD 3-7 UNITS PER AC',
 'RES MARSH MD 8-19 UNITS PER AC',
 'RES MARSH RURAL 2 OR LESS UNITS PER AC',
 'RES MD 8-19 UNIT

In [53]:
all_uses = list(df.landuse_1.unique()) + list(df.landuse_2.unique()) + list(df.landuse_3.unique()) \
           + list(df.landuse_4.unique()) + list(df.landuse_5.unique())
unique_uses = []
for use in all_uses:
    if use not in unique_uses and use != 'NA':
        unique_uses.append(use)
print(sorted(unique_uses))
print(len(unique_uses))

['AGR I - 1 UNIT PER 100 ACRES', 'AGR II - 1 UNIT PER 40 ACRES', 'AGR III - 1 UNIT PER 10 ACRES', 'AGR III MARSH - 1 UNIT PER 10 ACRES', 'AGR IV - 1 UNIT PER 2.5 ACRES', 'AGR IV CANAL - 1 UNIT PER 2.5 ACRES', 'AGR IV MARSH - 1 UNIT PER 2.5 ACRES', 'AGR IV RIVER - 1 UNIT PER 2.5 ACRES', 'BORROW PIT & DRAIN RESERV', 'BUSINESS PARK 1', 'COMM/RES/OFF', 'COMMERCIAL', 'COMMERCIAL WATERFRONT', 'CONSERVATION EASEMENT', 'EXCESS LAND', 'HEAVY INDUSTRIAL', 'INSTITUTIONAL', 'ISLAND', 'JURISDICTIONAL WETLANDS', 'LANDFILLS & DUMPS SOH', 'LIGHT INDUSTRIAL', 'MARSH', 'MH MD 8-19 UNITS PER AC', 'MIXED-USE PREDOM COMMERCIAL', 'MIXED-USE PREDOM RES', 'OFFICE', 'POND / LAKE', 'PRIVATE EASEMENTS', 'RES CANAL LD 3-7 UNITS PER AC', 'RES CANAL MD 8-19 UNITS PER AC', 'RES CANAL RURAL 2 OR LESS UNITS PER AC', 'RES GOLF LD 3-7 UNITS PER AC', 'RES GOLF MD 8-19 UNITS PER AC', 'RES HD 20-60 UNITS PER AC', 'RES LD 3-7 UNITS PER AC', 'RES MARSH LD 3-7 UNITS PER AC', 'RES MARSH MD 8-19 UNITS PER AC', 'RES MARSH RURAL 

In [ ]:
lists = {}
for item in sorted(unique_uses):
    lists[item] = []
for i in range(len(list(lists.keys()))):
    k = list(lists.keys())[i]
    print(k)
    for n in range(len(df)):
        if n % 20000 == 0:
            print(n)
        if df['landuse_1'][n] == k:
            lists[k] = lists[k] + [df['land_val_1'][n]]
        elif df['landuse_2'][n] == k:
            lists[k] = lists[k] + [df['land_val_2'][n]]
        elif df['landuse_3'][n] == k:
            lists[k] = lists[k] + [df['land_val_3'][n]]
        elif df['landuse_4'][n] == k:
            lists[k] = lists[k] + [df['land_val_4'][n]]
        elif df['landuse_5'][n] == k:
            lists[k] = lists[k] + [df['land_val_5'][n]]
        else:
            lists[k] = lists[k] + [0]

AGR I - 1 UNIT PER 100 ACRES
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
AGR II - 1 UNIT PER 40 ACRES
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
AGR III - 1 UNIT PER 10 ACRES
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
AGR III MARSH - 1 UNIT PER 10 ACRES
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
AGR IV - 1 UNIT PER 2.5 ACRES
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
AGR IV CANAL - 1 UNIT PER 2.5 ACRES
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
AGR IV MARSH - 1 UNIT PER 2.5 ACRES
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
AGR IV RIVER - 1 UNIT PER 2.5 ACRES
0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
BORROW PIT & DRAIN RESERV
0
20000
40000
60000
80000
100000
120000
140000
160000
18